# Průzkum tempdb
Následující příklad slouží k průzkumu vlastností a chování databáze tempdb.<br/>
1. Diagnostika pomocí DMV
2. Nedokumentovaný průzkum pomocí DBCC
3. Diagnostika PAGELATCH_ čekání pomocí XE

## 1. Diagnostika pomocí DMV
Trvalá databáze vs. tempdb

In [0]:
use msdb
select * from sys.dm_db_file_space_usage
GO
use tempdb
select * from sys.dm_db_file_space_usage
GO

V předchozí ukázce je vidět, že výsledek předchozích dotazů se liší ve sloupcích *version_store_...*, *user_object_...* a *internal_object_...*

## 2. Diagnostika pomocí DBCC funkcí
Ne všechny DBCC jsou dokumentované. Pro průzkum tempdb (nebo kterékoliv jiné databáze) lze použít DBCC PAGE funkci. Pro nápovědu k funkci DBCC PAGE je nutno použít TF 2588, pro zobrazení výsledku DBCC PAGE (nebo dalších nedokumentovaných DBCC funkcí) je nutno zapnout TF 3604.

In [0]:
use tempdb

dbcc help('?')
dbcc traceon(2588)  -- plny seznam a napovedy k DBCC
dbcc help('?')
dbcc help('page')

dbcc page(2,1,1,3)	-- PFS
dbcc traceon(3604)
dbcc page(2,1,1,3)	-- PFS


-- DMF pro ziskani alokaci pro jednotlive objekty
select * from sys.dm_db_database_page_allocations(2, null, null, null, null)   

-- konec
dbcc traceoff(3604)

## 3. Diagnostika PAGELATCH_ čekání pomocí XE
Extended Events (XE) jsou velmi silný diagnostický nástroj, ale je třeba si na ně zvyknout. Následující kód ukazuje jak založit XE session a jak ji dále rozebrat.

In [0]:
CREATE EVENT SESSION [pagelach on tempdb] ON SERVER 
ADD EVENT sqlos.wait_info(
    ACTION(sqlserver.database_id,sqlserver.sql_text)
    WHERE ([sqlserver].[database_id]=(2)))
ADD TARGET package0.event_file(SET filename=N'D:\temp\pagelatch.xel')
GO

ALTER EVENT SESSION [pagelach on tempdb] ON SERVER STATE = START

-- muze nasledovat simulace zateze

ALTER EVENT SESSION [pagelach on tempdb] ON SERVER STATE = STOP

XE session vytvořena, zachytává událost "wait info", která nastane v databázi tempdb (viz filtr)

Následuje rozbor získaného logu

In [0]:
-- #xel temp. table is used instead of the [xel] CTE to speed up the initial processing
drop table if exists #xel
select * 
	, cast(event_data as xml) as xml_event_data
into #xel
from sys.fn_xe_file_target_read_file('d:\temp\pagelatch_0_13195644*.xel', null, null, null)

select top 1000 * from #xel

--;with xel as (
--select * 
--	, cast(event_data as xml) as xml_event_data
--from sys.fn_xe_file_target_read_file('d:\temp\pagelatch_0_131956*.xel', null, null, null)
--)
--, 

;with shred as
(
select
	xel.*
	, c.value('(data[@name="wait_type"]/text)[1]', 'nvarchar(50)') as wait_type
	, c.value('(data[@name="wait_resource"]/value)[1]', 'nvarchar(50)') as wait_res
	, c.value('(action[@name="sql_text"]/value)[1]', 'nvarchar(4000)') as sql_text
	, c.value('(data[@name="opcode"]/text)[1]', 'nvarchar(10)') as opcode
	, c.value('(data[@name="duration"]/value)[1]', 'int') as duration
	, c.value('(data[@name="signal_duration"]/value)[1]', 'int') as signal_duration
	, c.value('(action[@name="database_id"]/value)[1]', 'int') as database_id
from #xel as xel
	cross apply xml_event_data.nodes('event') as t(c)
)
select *
from shred
where wait_type like 'page%latch%'
	and opcode = 'End'
	--and sql_text like '%\#%' {escape '\'}